## Automated QA platform analysis

In [129]:
# import the required libraries
import os
import sys

import numpy as np
import pandas as pd

In [130]:
# Change the file path to be same as that for which stats need to be generated

# gt_df = pd.read_excel('data/QA ground truth.xlsx')
# gt_df  = pd.read_excel('data/QA ground truth_27_03_2021.xlsx')    # Used in paper submission
# gt_df  = pd.read_excel('data/QA ground truth_29_04_2021.xlsx')
gt_df  = pd.read_excel('data/QA ground truth_19_06_2021.xlsx')   # Used in camera-ready paper

In [131]:
gt_df = gt_df[gt_df['Ground truth theme'].fillna('').apply(lambda x: x!='')]

### Data statistics

In [132]:
# Import the other required libraries

import bisect
import json
import os
import re
import statistics
import sys

import numpy as np
import pandas as pd
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
palette = sns.color_palette("bright", 4)
sns.set_style("whitegrid", {'axes.grid' : True})

from collections import Counter
from matplotlib.colors import ListedColormap
from matplotlib import pyplot

In [133]:
# Get the number of questions asked by the callers

from collections import defaultdict

dates = gt_df['Survey_date'].astype(str).tolist()
dates = [i.split(' ')[0] for i in dates]
caller_ids = gt_df['Callerid'].astype(str).tolist()
quess = gt_df['Query item id'].dropna().tolist()

print('{} callers asked {} questions'.format(len(set(caller_ids)), len(quess)))

dates_callers = defaultdict(list)

for date,caller_id in zip(dates, caller_ids):
    dates_callers[date].append(caller_id)
    
# for k,v in dates_callers.items():
#     print('{}->{}'.format(k,len(set(v))))
    
# print(dates_callers)
# print(len(dates))

241 callers asked 438 questions


In [134]:
from collections import Counter

date_cnt = Counter()

for dt in dates_callers:
    date_cnt[dt]+=1
    
sorted_dt_cnt = {k:v for k, v in sorted(date_cnt.items())}
    
print(sorted_dt_cnt)

{'2021-02-19': 1, '2021-02-22': 1, '2021-02-23': 1, '2021-02-24': 1, '2021-02-25': 1, '2021-02-26': 1, '2021-02-27': 1, '2021-02-28': 1, '2021-03-01': 1, '2021-03-04': 1, '2021-03-05': 1, '2021-03-06': 1, '2021-03-07': 1, '2021-03-08': 1, '2021-03-09': 1, '2021-03-10': 1, '2021-03-11': 1, '2021-03-12': 1, '2021-03-13': 1, '2021-03-14': 1, '2021-03-15': 1, '2021-03-16': 1, '2021-03-17': 1, '2021-03-18': 1, '2021-03-19': 1, '2021-03-20': 1, '2021-03-21': 1, '2021-03-22': 1, '2021-03-23': 1, '2021-03-24': 1, '2021-03-25': 1, '2021-03-26': 1, '2021-03-27': 1, '2021-03-28': 1, '2021-03-29': 1, '2021-03-30': 1, '2021-03-31': 1, '2021-04-01': 1, '2021-04-02': 1, '2021-04-04': 1, '2021-04-05': 1, '2021-04-06': 1, '2021-04-07': 1, '2021-04-08': 1, '2021-04-09': 1, '2021-04-10': 1, '2021-04-11': 1, '2021-04-12': 1, '2021-04-13': 1, '2021-04-14': 1, '2021-04-15': 1, '2021-04-16': 1, '2021-04-17': 1, '2021-04-18': 1, '2021-04-19': 1, '2021-04-20': 1, '2021-04-22': 1, '2021-04-23': 1, '2021-04-24':

#### Overall User statistics

In [135]:
from collections import Counter

def get_overall_stats(fdf):
    unique_callers = set(fdf['Callerid'].tolist())
    print("Number of unique callers : {}".format(len(unique_callers)))
    
    tot_ques = len(fdf)
    print("Total number of questions asked : {}".format(tot_ques))
    
    themes_selected = fdf['Selected theme'].str.strip().fillna('default').tolist()
    themes_selected = [i if i!='' else 'default' for i in themes_selected]
    fdf['Selected theme'] = themes_selected
#     print(fdf['Selected theme'].unique().tolist())
    
    theme_cnt = Counter()
    for theme in themes_selected:
        theme_cnt[theme] += 1
    
    tot_theme_select = sum(list(theme_cnt.values()))
    def_cnt = theme_cnt['default']
    non_def_cnt = tot_theme_select - def_cnt
    
#     print(theme_cnt)
#     print(tot_theme_select)

    print("{}% of people selected a theme".format((non_def_cnt/tot_theme_select)*100))
    print("{}% of people selected default theme".format((def_cnt/tot_theme_select)*100))
    
    gt_themes = fdf['Ground truth theme'].astype(str).tolist()
    ques_cnt = Counter()
    for gt_theme in gt_themes:
        ques_cnt[gt_theme] += 1
        
    tot_ques_cnt = sum(ques_cnt.values())
    no_ques_asked = ques_cnt['<no question>']
    not_rel_ques = ques_cnt['<not relevant>']
    ques_asked = tot_ques_cnt-no_ques_asked-not_rel_ques
    
    print("Number of relevant questions {} out of {}".format(ques_asked, tot_ques_cnt))
    
    print("{}% of people asked a question".format((ques_asked/tot_ques_cnt)*100))
    print("{}% of people did not ask question".format((no_ques_asked/tot_ques_cnt)*100))
    print("{}% of people did not ask a relevant question".format((not_rel_ques/tot_ques_cnt)*100))
    
#     print(tot_ques_cnt)
    print(ques_cnt)

In [136]:
get_overall_stats(gt_df)

Number of unique callers : 241
Total number of questions asked : 438
91.55251141552512% of people selected a theme
8.447488584474886% of people selected default theme
Number of relevant questions 156 out of 438
35.61643835616438% of people asked a question
63.013698630136986% of people did not ask question
1.36986301369863% of people did not ask a relevant question
Counter({'<no question>': 276, 'coronavirus': 53, 'matruswasthyaposhan': 26, 'shishupaalanposhan': 26, 'mahvaari': 22, 'anyaswasthyajaankari': 18, 'chamkibukhar': 11, '<not relevant>': 6})


#### Theme Pie-chart

In [137]:
import plotly.express as px
from collections import Counter

themes = gt_df['Selected theme'].str.strip().fillna('default').tolist()
themes = [i if i!='' else 'default' for i in themes]

themes_cnt = Counter()

for theme in themes:
    themes_cnt[theme]+=1
print(themes_cnt)

theme_map = {'matruswasthyaposhan': 'Maternal health & nutrition', 'coronavirus': 'Coronavirus',
             'mahvaari': 'Menstruation', 'shishupaalanposhan':'Child care & nutrition',
             'anyaswasthyajaankari':'Others', 'chamkibukhar':'AES', 'default':'Default'}

eng_themes_cnt = {}

for theme in themes_cnt:
    eng_themes_cnt[theme_map[theme]] = themes_cnt[theme]
    
print(eng_themes_cnt)

# pie_df = pd.DataFrame({'Themes':list(themes_cnt.keys()), 'Counts':list(themes_cnt.values())})
pie_df = pd.DataFrame({'Themes':list(eng_themes_cnt.keys()), 'Counts':list(eng_themes_cnt.values())})
fig = px.pie(pie_df, values='Counts', names='Themes', color_discrete_sequence=px.colors.sequential.Viridis, hole=0.7)
fig.update_layout(
#     title='Date wise normalized distribution of issues in '+ req_st,
    width=900, height=500,
    showlegend=False)
fig.update_traces(textinfo='label+percent', textposition='outside', textfont_size=16)
fig.show()

Counter({'matruswasthyaposhan': 144, 'mahvaari': 72, 'coronavirus': 67, 'shishupaalanposhan': 60, 'default': 37, 'anyaswasthyajaankari': 33, 'chamkibukhar': 25})
{'Coronavirus': 67, 'AES': 25, 'Others': 33, 'Default': 37, 'Child care & nutrition': 60, 'Maternal health & nutrition': 144, 'Menstruation': 72}


In [138]:
fig.write_image("plots/themes_pie_chart_new.png")

#### User satisfaction

In [146]:
from collections import Counter

def get_sat_users(fdf, col_name):
    sat_users = fdf[col_name].fillna('').tolist()
    sat_cnt = Counter()

    for sat in sat_users:
        sat_cnt[sat.strip().lower()] += 1
    
    print(sat_cnt)
    
    print((sat_cnt['yes']/(sat_cnt['yes']+sat_cnt['no']))*100)
    
    tot = sum(sat_cnt.values())
    sat_perc = (sat_cnt['yes']/tot)*100
    unsat_perc = (sat_cnt['no']/tot)*100
    no_ans_perc = (sat_cnt['']/tot)*100
    
    return sat_perc, unsat_perc, no_ans_perc

def info_seekers_sat(fdf, col_name='User Satisfied'):
    info_seekers_df = fdf[fdf['Ground truth theme'].apply(lambda x: x!='<no question>' and x!='<not relevant>')]
    non_info_seekers_df = fdf[fdf['Ground truth theme'].apply(lambda x: x=='<no question>' or x=='<not relevant>')]
    
    info_sat_perc, info_unsat_perc, info_no_ans_perc = get_sat_users(info_seekers_df, col_name)
    non_info_sat_perc, non_info_unsat_perc, non_info_no_ans_perc = get_sat_users(non_info_seekers_df, col_name)
    
    print(info_sat_perc, info_unsat_perc, info_no_ans_perc)
    print(non_info_sat_perc, non_info_unsat_perc, non_info_no_ans_perc)

In [147]:
sat_perc, unsat_perc, no_ans_perc = get_sat_users(gt_df, 'User Satisfied')
print(sat_perc, unsat_perc, no_ans_perc)

Counter({'yes': 199, '': 188, 'no': 51})
79.60000000000001
45.4337899543379 11.643835616438356 42.922374429223744


In [148]:
re_ask, not_reask, no_ans_reask = get_sat_users(gt_df, 'Reasked (yes/no)')
print(re_ask, not_reask, no_ans_reask)

Counter({'': 198, 'yes': 183, 'no': 57})
76.25
41.78082191780822 13.013698630136986 45.20547945205479


In [149]:
info_seekers_sat(gt_df)

Counter({'yes': 101, '': 34, 'no': 21})
82.78688524590164
Counter({'': 154, 'yes': 98, 'no': 30})
76.5625
64.74358974358975 13.461538461538462 21.794871794871796
34.751773049645394 10.638297872340425 54.60992907801418


In [150]:
info_seekers_sat(gt_df, 'Reasked (yes/no)')

Counter({'yes': 88, '': 42, 'no': 26})
77.19298245614034
Counter({'': 156, 'yes': 95, 'no': 31})
75.39682539682539
56.41025641025641 16.666666666666664 26.923076923076923
33.687943262411345 10.99290780141844 55.319148936170215


In [155]:
from collections import Counter
def info_seekers_sat_reask(fdf):
    info_seekers_df = fdf[fdf['Ground truth theme'].apply(lambda x: x!='<no question>' and x!='<not relevant>')]
#     non_info_seekers_df = fdf[fdf['Ground truth theme'].apply(lambda x: x=='<no question>' or x=='<not relevant>')]
    info_seek_not_sat_df = info_seekers_df[info_seekers_df['User Satisfied'].fillna('').apply(lambda x: x.lower()=='no')]
    
    info_sat_perc, info_unsat_perc, info_no_ans_perc = get_sat_users(info_seek_not_sat_df, 'Reasked (yes/no)')
#     info_sat_perc, info_unsat_perc, info_no_ans_perc = get_sat_users(info_seekers_df, col_name)
#     non_info_sat_perc, non_info_unsat_perc, non_info_no_ans_perc = get_sat_users(non_info_seekers_df, col_name)
    
    print(info_sat_perc, info_unsat_perc, info_no_ans_perc)
#     print(non_info_sat_perc, non_info_unsat_perc, non_info_no_ans_perc)

In [156]:
info_seekers_sat_reask(gt_df)

Counter({'yes': 10, 'no': 8, '': 3})
55.55555555555556
47.61904761904761 38.095238095238095 14.285714285714285


### Error Analysis

In [160]:
def get_thematic_err(fdf):
    # Remove the questions without ground truth
    fdf = fdf[fdf['Ground truth theme'].fillna('').apply(lambda x: x!='')]
#     print(len(fdf))
    gt_themes = fdf['Ground truth theme'].str.strip().fillna('').tolist()
    sel_themes = fdf['Selected theme'].str.strip().fillna('default').tolist()

    tot_ques = len(fdf)
    print(tot_ques)
    
#     sel_themes = [i.strip() for i in sel_themes]
#     gt_themes = [i.strip() for i in gt_themes]
    
    print(set(gt_themes))    # just for sanity check
    print(set(sel_themes))   # just for sanity check
    
    corr_theme_cnt = 0
    def_theme_cnt = 0
    incorr_theme_cnt = 0
    def_ques_cnt = 0
    
    for gt_theme, sel_theme in zip(gt_themes, sel_themes):
        if sel_theme == 'default':
            def_theme_cnt += 1
        else:
            if gt_theme == '<no question>':
                def_ques_cnt += 1
            else:
                if gt_theme != '<not relevant>':
                    if gt_theme == sel_theme:
                        corr_theme_cnt += 1
                    else:
                        incorr_theme_cnt += 1
    
    print("Total ques : {}".format(tot_ques))
#     print(def_ques_cnt)
    print("{:.2f}% of people did not select theme and hence went to default theme".format((def_theme_cnt/tot_ques)*100))
    print("{:.2f}% of people selected a theme but asked question of another tmatruswasthyaposhanheme".format((incorr_theme_cnt/tot_ques)*100))
    print("{:.2f}% of people selected a theme and asked question from that theme ".format((corr_theme_cnt/tot_ques)*100))
    print("{:.2f}% of people selected a theme but did not ask a question".format((def_ques_cnt/tot_ques)*100))
#     print(corr_theme_cnt/tot_ques)
#     print(def_theme_cnt/tot_ques)
#     print(incorr_theme_cnt/tot_ques)

In [161]:
get_thematic_err(gt_df)

438
{'matruswasthyaposhan', 'mahvaari', '<not relevant>', 'coronavirus', '<no question>', 'anyaswasthyajaankari', 'shishupaalanposhan', 'chamkibukhar'}
{'matruswasthyaposhan', 'mahvaari', 'coronavirus', 'default', 'anyaswasthyajaankari', 'shishupaalanposhan', 'chamkibukhar'}
Total ques : 438
8.45% of people did not select theme and hence went to default theme
5.71% of people selected a theme but asked question of another tmatruswasthyaposhanheme
29.68% of people selected a theme and asked question from that theme 
55.02% of people selected a theme but did not ask a question


In [162]:
def get_dataset_err(fdf):
    fdf = fdf[fdf['Ground truth sanitized question ID'].fillna('').apply(lambda x: x!='')]
    tot_cnt = len(fdf)
    ques_ids = fdf['Ground truth sanitized question ID'].tolist()
    print(fdf['Ground truth sanitized question ID'].tolist()) # sanity check
    data_err_cnt = 0
    for ques_id in ques_ids:
        if 'not available' in str(ques_id).lower():
            data_err_cnt += 1
    print("{:.2f}% of questions were out of data".format((data_err_cnt/tot_cnt)*100))

In [163]:
get_dataset_err(gt_df)

[2375402, 'Not available', 'Not available', 'Not available', 2396578, 2375322, 'Not available', 'Not available', 2375497, 2428433, 'Not available', 2374688, 'Not available', 'Not available', 2371954, 'Not available', 2428131, 'Not available', 2375435, 2375435, 'Not available', 'Not available', 'Not available', 2375402, 2371954, 'Not available', 2371990, 2375647, 2372251, 2372223, 2372223, 2396578, 2372090, 2374705, 2375406, 2374682, 2375635, 'Not available', 2372122, 'Not available', 'Not available', 'Not available', 'Not available', 'Not available', 'Not available', 2371954, 'Not available', 2375488, 2375315, 'Not available', 2375402, 2371954, 2374688, 2375418, 'Not available', 2374705, 2428438, 2371934, 'Not available', 'Not available', 2375488, 2375488, 'Not available', 2375406, 2374733, 2469898, 2374733, 2375406, 'not available', 2469898, 'not available', 2375617, 'not available', 2374706, 2371957, 'Not available', 2374705, 2375630, 'Not available', 2375435, 2371954, 'not available

In [164]:
def get_model_err(fdf):
    fdf = fdf[fdf['Query answered by ML (yes/no)'].fillna('').apply(lambda x: x!='')]
    tot_cnt = len(fdf)
    
    ml_anss = fdf['Query answered by ML (yes/no)'].fillna('').tolist()
    gt_themes = fdf['Ground truth theme'].str.strip().fillna('').tolist()
    sel_themes = fdf['Selected theme'].str.strip().fillna('default').tolist()
    print(ml_anss)    # sanity check
    
    tot_err = 0
    ml_err_cnt = 0
    ml_err_with_theme_cnt = 0
    
    for ml_ans, gt_theme, sel_theme in zip(ml_anss, gt_themes, sel_themes):
        if ml_ans.lower() == 'no':
            tot_err += 1
            if gt_theme != sel_theme:
                ml_err_with_theme_cnt += 1
            else:
                ml_err_cnt += 1
    
    print("Combined error : {}".format((tot_err/tot_cnt)*100))
    print("Out of available answers, {:.2f}% answers were not caught by ML model because of gaps in model,\
{:.2f}% answers were not caught because user selected incorrect theme".format((ml_err_cnt/tot_cnt)*100,\
(ml_err_with_theme_cnt/tot_cnt)*100))

In [165]:
get_model_err(gt_df)

['Yes', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'no', 'No', 'yes', 'no', 'yes', 'yes', 'no', 'yes', 'yes', 'no', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'No', 'yes', 'No', 'yes', 'no', 'Yes', 'no', 'yes', 'no', 'yes', 'yes', 'no', 'yes', 'no', 'no', 'no', 'no', 'yes', 'no', 'yes', 'no', 'no', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'yes']
Combined error : 47.25274725274725
Out of available answers, 30.77% answers were not caught by ML model because of gaps in model,16.48% answers were not caught because user selected incorrect theme


In [110]:
def corr_ans_heard(fdf):
    fdf = fdf[fdf['Correct answer heard (yes/no)'].fillna('').apply(lambda x: x!='')]
    tot_cnt = len(fdf)
    
    ans_heard = fdf['Correct answer heard (yes/no)'].fillna('').tolist()
    print(ans_heard)    # Sanity check
    
    ans_not_heard_cnt = 0
    
    for ans in ans_heard:
        if ans.lower() == 'no':
            ans_not_heard_cnt += 1
    print("{:.2f}% users did not listen to the correct answer even when it was present in top 3".format((ans_not_heard_cnt/tot_cnt)*100))

In [111]:
corr_ans_heard(gt_df)

['No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'yes', 'Yes', 'Yes', 'No', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'no', 'yes', 'No', 'yes', 'yes', 'yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'no', 'no', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'no', 'no', 'no', 'yes', 'no', 'no', 'no', 'no']
49.15% users did not listen to the correct answer even when it was present in top 3


### Density plot for number of answers heard

In [120]:
penguins = sns.load_dataset("penguins")
penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [130]:
tips = sns.load_dataset("tips")
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [148]:
import plotly.figure_factory as ff
import numpy as np

x1 = np.random.randn(200) - 1
x2 = np.random.randn(200)
x3 = np.random.randn(200) + 1

hist_data = [x1, x2, x3]

group_labels = ['Group 1', 'Group 2', 'Group 3']
colors = ['#333F44', '#37AA9C', '#94F3E4']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels, show_hist=False, colors=colors)

# Add title
fig.update_layout(title_text='Curve and Rug Plot')
fig.show()

In [ ]:
def get_usage_plot(file_name, ml_sheet_names, non_ml_sheet_names, trans_type, col_name):
    ml_df = pd.DataFrame({})
    non_ml_df = pd.DataFrame({})
    
    # Concatenate the ml/non-ml sheets
    for sheet_name in ml_sheet_names:
        filedf = pd.read_excel(file_name, sheet_name=sheet_name)
        ml_df = pd.concat([ml_df, filedf])
    for sheet_name in non_ml_sheet_names:
        filedf = pd.read_excel(file_name, sheet_name=sheet_name)
        non_ml_df = pd.concat([non_ml_df, filedf])
        
    ml_df = ml_df[ml_df['ML State']!='REJ']
    
#     if col_name == 'Moderation Time':
    ml_df = ml_df[ml_df['ACC/REJ'].apply(lambda x: str(x).lower().strip()!='rej')]
    non_ml_df = non_ml_df[non_ml_df['ACC/REJ'].apply(lambda x: str(x).lower().strip()!='rej')]
    
    # Extra check
    ml_df = ml_df[ml_df[col_name].fillna(0).apply(lambda x: x>0 and str(x)!='-')]
    non_ml_df = non_ml_df[non_ml_df[col_name].fillna(0).apply(lambda x: x>0 and str(x)!='-')]

    ml_plot_df = get_line_plot_df(ml_df, 'ML', col_name)
    non_ml_plot_df = get_line_plot_df(non_ml_df, 'Non-ML', col_name)
    
    get_perc_reduction(ml_plot_df, non_ml_plot_df, col_name)
    
    line_plot_df = pd.concat([ml_plot_df, non_ml_plot_df])
    
    # Convert the index column into normal column
    line_plot_df['Item Duration (in seconds)']=line_plot_df.index
    line_plot_df = line_plot_df.rename(columns={col_name: '{} (in seconds)'.format(col_name)})
    
    ax = sns.lineplot(data=line_plot_df, x="Item Duration (in seconds)", y='{} (in seconds)'.format(col_name), hue="Type", style='Type', markers=True, err_style="bars", ci=68).set_title(trans_type)
    return ax

In [ ]:
get_usage_plot()